In [13]:
import os
import string

import networkx as nx
import nltk
from nltk.util import ngrams

In [14]:
graph = nx.read_gexf('out/elbulli_dat.gexf')

In [15]:
g_nlg = nx.read_gexf('out/elbulli_nlg.gexf')

In [4]:
all_ingredients_graph = nx.read_gexf('data/spanish_ingredients_lexicon_5.gexf')

In [ ]:
ingredients = all_ingredients_graph.nodes()

In [5]:
len(ingredients)

26743

In [9]:
ingredients[:10]

['jugos apio',
 'chicharros',
 'jengibres deshidratado',
 'láminas de papas',
 'churrascos ternera',
 'lomo de puerco',
 'esmeraldas limon',
 'jugos de pollos',
 'hongo hongo alga nori sequias',
 'obleas de arroces']

In [10]:
all_techniques_graph = nx.read_gexf('data/spanish_techniques_lexicon_5.gexf')

In [ ]:
techniques = all_techniques_graph.nodes()

In [11]:
len(techniques)

4372

In [9]:
techniques[:10]

['retencion calorizado',
 'al hornagueamos presionado',
 'sofreír a presionado',
 'al grilla presionado',
 'cocidos al vapora',
 'productos horneados a presion',
 'cocinando vaciando',
 'en jiangsu jugamos rui',
 'repostado',
 'salteando de microondas']

In [64]:
def trim(s):
    return ' '.join(s.split())

In [11]:
def my_ngrams(technique):
    ngrms = []
    tokens = nltk.word_tokenize(technique)
    for i in range(1, len(tokens) + 1):
        ngrms.extend(ngrams(tokens, i))
    return list(map(lambda x: ' '.join(x), ngrms))

In [62]:
for n, data in g_nlg.nodes(data=True):
    if data['nodetype'] == 'Producto':
        print(n)
        print(data)
        break

Producto:mozzarella de búfala   crema doble
{'label': 'Producto:mozzarella de búfala   crema doble', 'nodetype': 'Producto'}


In [13]:
%%time



# path = 'data/recipes/elbulli/'
# for folder in os.listdir(path):
#     for filename_number in sorted(map(lambda x: int(x[:-4]), os.listdir(path + folder))):
#         filename = str(filename_number) + '.dat'
#         with open(path + folder + '/' + filename) as f:
#             row = {
#                 '_id': '',
#                 'title': '',
#                 'year': 0,
#                 'ingredients': set(),
#                 'techniques': set(),
#             }
#             ingreds = set()
#             techns = set()
#             for line in f:
#                 line = line.lower().strip()
#                 if line.startswith('num'):
#                     ide = line.split('=')[1]
#                 elif line.startswith('&titol='):
#                     title = line.split('=')[1]
#                 elif line.startswith('&any'):
#                     year = int(line.split('=')[1])
#                 elif line.startswith('&ingredientselaboracio'):
#                     elems = line.split('=')[1].split('#')
#                     ingrs = map(trim, elems)
#                     for ingr in ingrs:
#                         ngrms = my_ngrams(ingr)
#                         ngrms.reverse()
#                         for ngrm in ngrms:
#                             if ngrm in ingredients:
#                                 ingreds.add(ngrm)
#                                 all_ingredients_graph.node[ngrm]['count'] += 1
#                                 break
#                 elif line.startswith('&descripcioelaboracio') or \
#                      line.startswith('&acabatipresentacio') or \
#                      line.startswith('&titolelaboracio'):
#                     elems = line.split('=')[1].split('#')
#                     steps = map(trim, elems)
#                     for step in steps:
#                         used_ngrams = set()
#                         ngrms = my_ngrams(step)
#                         ngrms.reverse()
#                         for ngrm in ngrms:
#                             if ngrm in techniques and all(map(lambda x: ngrm not in x, used_ngrams)):
#                                 techns.add(ngrm)
#                                 all_techniques_graph.node[ngrm]['count'] += 1
#                                 used_ngrams.add(ngrm)
#             row['_id'] = ide
#             row['title'] = title.lower()
#             row['year'] = year
#             row['ingredients'] = list(ingreds)
#             row['techniques'] = list(techns)
#             rows.append(row)
            
#             count += 1
#             if count % 100 == 0:
#                 db.elbulli_raw.insert_many(rows)
#                 rows = []
#                 print(count, 'rows inserted')
# db.elbulli_raw.insert_many(rows)
# rows = []
# print(count, 'rows inserted')

# CPU times: user 26min 38s, sys: 560 ms, total: 26min 38s
# Wall time: 26min 37s

In [49]:
prep_prod_rels = ['bañado', 'alcohol', 'chocolate', 'lacteo', 'nuevaPasta', 'producto', 'relleno']
prep_ingr_rels = ['composicion']
prep_flav_rels = ['sabor']

def get_prep_products(g, prep):
    return [k for k in g[prep] if g[prep][k]['edgetype'] in prep_prod_rels]

def get_prep_ingredients(g, prep):
    return [k for k in g[prep] if g[prep][k]['edgetype'] in prep_ingr_rels]

def get_prep_flavors(g, prep):
    return [k for k in g[prep] if g[prep][k]['edgetype'] in prep_flav_rels]

def get_recip_preparations(g, recip):
    return [k for k in g[recip] if g[recip][k]['edgetype'] == 'elaboracion']

def get_recip_products(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_products(g, prep)}

def get_recip_ingredients(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_ingredients(g, prep)}

def get_recip_flavors(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_flavors(g, prep)}

def get_recip_components(g, recip):
    prods = get_recip_products(g, recip)
    ingrs = get_recip_ingredients(g, recip)
    flavs = get_recip_flavors(g, recip)
    return prods.union(ingrs).union(flavs)